<div style="text-align: left;"><img src="https://www.juliabox.org/assets/img/juliacloudlogo.png" style="margin: 0px 0px 0px 0px; padding-right: 20px;width: 80px; float: left;" title="" alt="" /></div>
<img src="http://dmkpress.com/images/cms/thumbs/a5b0aeaa3fa7d6e58d75710c18673bd7ec6d5f6d/978-5-97060-370-3_270_369__100.jpg" style="margin: 0px 0px 5px 20px; width: 100px; float: right;" title="" alt="" />
Всестороннее введение в новый язык программирования для научно-технических вычислений [Julia](http://julialang.org/) в книге Малколма Шеррингтона, Packt Publishing, июль 2015.

<h1>Осваиваем язык Julia</h1><br />

Совершенствование мастерства в области аналитики и программирования при помощи Julia в целях решения задач комплексной обработки данных
<div style="text-align: left;font-size:8pt;padding-top:10px;">Программный код Julia (v0.4.5) протестирован в Windows 8.1/10 и Linux/Lubuntu 16.4</div>
<div style="text-align: left;"><h1>Глава 6. Научное программирование</h1></div>

# Стохастические задачи
## Стохастическое моделирование
### Пакет SimJulia
### Пример с клиентским обслуживанием в банке

Охватывает:

<ul><li>Ресурсы</li>
<li>Операторы событий</li>
<li>Операционист(ы) со случайным временем обслуживания и клиенты, которые могут отказаться от услуги</li></ul>

Пример моделирует банковских операционистов и клиентов, прибывающих в случайные моменты времени. Каждый клиент имеет определенное терпение. Он ждет, чтобы получить услугу, пока показатель терпения не превысит установленный максимум. Если клиент добирается до операциониста, то он занимает ресурс в течение некоторого времени, прежде чем тот освободится.
Новые клиенты создаются с помощью процесса источника через заданные промежутки времени.

In [67]:
using SimJulia
using Distributions

function customer(env::Environment, name::UTF8String, counter::Resource, time_in_bank::Float64)
  arrive = now(env)
    
  println("$arrive $name: прибыл")
    
  req = Request(counter)
  patience = rand(Uniform(MIN_PATIENCE, MAX_PATIENCE))
  result = yield(req | Timeout(env, patience))
    
  wait = now(env) - arrive
  push!(wait_times, wait);
    
  if in(req, keys(result))
    println("$(now(env)) $name: прождал $wait") 
    yield(Timeout(env, rand(Exponential(time_in_bank))))
    println("$(now(env)) $name: обслужен")    
    yield(Release(counter))
    #push!(in_bank_times, now(env) - arrive);
  else
    println("$(now(env)) $name: ОТКАЗАЛСЯ после $wait")
    push!(renege_qty, 1);       
    SimJulia.cancel(req)
  end
end

function source(env::Environment, number::Int, interval::Float64, counter::Resource)
  d = Exponential(interval)
  for i in 1:number
    Process(env, customer, "Клиент №$i", counter, 12.0)
    yield(Timeout(env, rand(d)))
  end
end

function randomize()  # lcg_rand
  seed = ccall( (:GetTickCount, "kernel32"), stdcall, Int32, () );
  ((seed * 214013 + 2531011) >> 16) & 0x7fff
end

# Настроить и запустить имитационный прогон
println("Отказы от услуги в банке\n")

# Установить исходные данные имитации 

const MAX_CUSTOMERS     = 20    # Общее число клиентов
const MAX_TIME          = 300.0 # Общее время работы банка
const MEAN_ARRIVAL_TIME = 5.0   # Генерировать новых клиентов с прим. промежутком x (интервалы)
const MIN_PATIENCE      = 1.0   # Минимальное терпение клиента
const MAX_PATIENCE      = 8.0   # Максимальное терпение клиента (срд.время обслуживания)
const MAX_TELLERS       = 2     # Число операционистов

# Использовать ранее определенную функцию randomize() либо 
# выбрать значение, чтобы сделать прогон воспроизводимым
#const RANDOM_SEED = 150  
const RANDOM_SEED = randomize()
srand(RANDOM_SEED)

global wait_times = Float64[];  # Времена ожидания
global renege_qty = Int32[];    # Количество отказов

env = Environment()

# Запустить процессы и выполнить
counters = Resource(env, MAX_TELLERS)
Process(env, source, MAX_CUSTOMERS, MEAN_ARRIVAL_TIME, counters)
run(env, MAX_TIME)

@printf "\nПри наличии %s операциониста(ов):\n" MAX_TELLERS
@printf "Среднее время ожидания = %5.3f\n" mean(wait_times)
@printf "Количество отказов = %5d" sum(renege_qty)

Отказы от услуги в банке

0.0 Клиент №1: прибыл
0.0 Клиент №1: прождал 0.0
11.218333487073936 Клиент №1: обслужен
11.227563640488105 Клиент №2: прибыл
11.227563640488105 Клиент №2: прождал 0.0
27.74817838703681 Клиент №3: прибыл
27.74817838703681 Клиент №3: прождал 0.0
34.065360123996435 Клиент №4: прибыл
38.638123560112064 Клиент №4: ОТКАЗАЛСЯ после 4.572763436115629
39.858414534348455 Клиент №5: прибыл
42.8863217661514 Клиент №6: прибыл
44.0321199479289 Клиент №5: ОТКАЗАЛСЯ после 4.173705413580443
44.65080202644407 Клиент №7: прибыл
48.45277440135108 Клиент №8: прибыл
49.15620363932055 Клиент №3: обслужен
49.15620363932055 Клиент №6: прождал 6.269881873169155
49.99565075924825 Клиент №7: ОТКАЗАЛСЯ после 5.344848732804181
50.921042991008285 Клиент №6: обслужен
50.921042991008285 Клиент №8: прождал 2.4682685896572067
60.561417512062036 Клиент №9: прибыл
65.93649345216706 Клиент №9: ОТКАЗАЛСЯ после 5.37507594010502
70.05777885687267 Клиент №8: обслужен
71.81242442325122 Клиент №10: приб

Количество отказов =     6

# Байесовские методы и марковские процессы
## Методы Монте-Карло по схеме марковских цепей
### Программные платформы на основе MCMC

In [3]:
cd(joinpath(homedir(),"julia_projects","data"))

using Mamba

dogs = (Symbol => Any)[:Y => round(Int, readdlm("dogs.wsv")) ]

dogs[:Dogs] = size(dogs[:Y], 1)
dogs[:Trials] = size(dogs[:Y], 2)
dogs[:xa] = mapslices(cumsum, dogs[:Y], 2)
dogs[:xs] = mapslices(x -> collect(1:25) - x, dogs[:xa], 2)
dogs[:y] = 1 - dogs[:Y][:, 2:25]


model = Model(       # Спецификация модели
  y = Stochastic(2,
    (Dogs, Trials, alpha, xa, beta, xs) ->
      UnivariateDistribution[
        begin
          p = exp(alpha * xa[i, j] + beta * xs[i, j])
          Bernoulli(p)
        end
        for i in 1:Dogs, j in 1:Trials-1
      ],
    false
  ),
  alpha = Stochastic(
    () -> Truncated(Flat(), -Inf, -1e-5)
  ),
  A = Logical(
    alpha -> exp(alpha)
  ),
  beta = Stochastic(
    () -> Truncated(Flat(), -Inf, -1e-5)
  ),
  B = Logical(
    beta -> exp(beta)
  )
)
inits = [   # Начальные значения
  Dict(:y => dogs[:y], :alpha => -1, :beta => -1),
  Dict(:y => dogs[:y], :alpha => -2, :beta => -2)
]


scheme = [Slice([:alpha, :beta], 1.0)]   # План выборки ([1.0, 1.0])
setsamplers!(model, scheme)

# Имитационный прогон по схеме MCMC
sim = mcmc(model, dogs, inits, 10000, burnin=2500, thin=2, chains=2)


MCMC Simulation of 10000 Iterations x 2 Chains...

Chain 1:   0% [0:31:36 of 0:31:38 remaining]
Chain 1:  10% [0:01:36 of 0:01:47 remaining]
Chain 1:  20% [0:01:18 of 0:01:37 remaining]
Chain 1:  30% [0:01:07 of 0:01:36 remaining]
Chain 1:  40% [0:00:57 of 0:01:34 remaining]
Chain 1:  50% [0:00:47 of 0:01:34 remaining]
Chain 1:  60% [0:00:37 of 0:01:33 remaining]
Chain 1:  70% [0:00:28 of 0:01:32 remaining]
Chain 1:  80% [0:00:18 of 0:01:32 remaining]
Chain 1:  90% [0:00:09 of 0:01:32 remaining]
Chain 1: 100% [0:00:00 of 0:01:32 remaining]

Chain 2:   0% [0:01:08 of 0:01:08 remaining]
Chain 2:  10% [0:01:23 of 0:01:33 remaining]
Chain 2:  20% [0:01:12 of 0:01:30 remaining]
Chain 2:  30% [0:01:04 of 0:01:32 remaining]
Chain 2:  40% [0:00:56 of 0:01:34 remaining]
Chain 2:  50% [0:00:46 of 0:01:33 remaining]
Chain 2:  60% [0:00:37 of 0:01:33 remaining]
Chain 2:  70% [0:00:29 of 0:01:38 remaining]
Chain 2:  80% [0:00:20 of 0:01:39 remaining]
Chain 2:  90% [0:00:10 of 0:01:38 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 2502:10000
Thinning interval = 2
Chains = 1,2
Samples per chain = 3750

3750x4x2 Array{Float64,3}:
[:, :, 1] =
 -0.0738941  0.92877   -0.238002  0.788201
 -0.0759568  0.926856  -0.247935  0.780411
 -0.0742569  0.928433  -0.266703  0.765901
 -0.0756774  0.927115  -0.239215  0.787245
 -0.0954462  0.908967  -0.226897  0.797003
 -0.104385   0.900878  -0.233161  0.792026
 -0.0780271  0.924939  -0.222208  0.800749
 -0.0835091  0.919883  -0.219682  0.802774
 -0.0671004  0.935101  -0.277139  0.757949
 -0.0763262  0.926514  -0.283058  0.753476
 -0.0845308  0.918943  -0.204978  0.814665
 -0.0862578  0.917358  -0.212734  0.808371
 -0.0796541  0.923436  -0.234131  0.791258
  ⋮                                       
 -0.0764972  0.926355  -0.263312  0.768502
 -0.0638957  0.938103  -0.272875  0.761188
 -0.0681673  0.934104  -0.260858  0.77039 
 -0.084014   0.919418  -0.229787  0.794703
 -0.0961432  0.908334  -0.244384  0.783187
 -0.0674776  0.934749  

In [4]:
describe(sim)

Iterations = 2502:10000
Thinning interval = 2
Chains = 1,2
Samples per chain = 3750

Empirical Posterior Estimates:
          Mean          SD        Naive SE        MCSE        ESS   
 beta -0.078932952 0.011925888 0.00013770829 0.00020372007 3426.9976
    B  0.924167537 0.011008323 0.00012711317 0.00018799514 3428.8550
alpha -0.244071223 0.024974572 0.00028838151 0.00042714180 3418.6288
    A  0.783675757 0.019525964 0.00022546642 0.00033409673 3415.7064

Quantiles:
          2.5%        25.0%       50.0%        75.0%        97.5%   
 beta -0.103340923 -0.08679858 -0.07855068 -0.070606809 -0.056836984
    B  0.901819470  0.91686176  0.92445521  0.931828207  0.944748066
alpha -0.295090768 -0.26067894 -0.24273318 -0.226682685 -0.197668558
    A  0.744464011  0.77052826  0.78448080  0.797173698  0.820641805



In [ ]:
cd(joinpath(homedir(),"julia_projects","images"))

using Gadfly

p = plot(sim)
draw(p, filename="figure_6_10_dog.svg")

# В поле STDIN> будет запрошено разрешение (Y/N) записать файл .SVG